In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
ext = 'drive/MyDrive/CV_Images/'#concentric.mat'

In [ ]:
! pip install imagehash

     |████████████████████████████████| 812 kB 4.9 MB/s 
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295206 sha256=4f6675c8d1efcd7d5bdbac264cb020fb1813b18e85e1748660955767a64ae2dc
  Stored in directory: /root/.cache/pip/wheels/4c/d5/59/5e3e297533ddb09407769762985d134135064c6831e29a914e
Successfully built imagehash


In [ ]:
!pip install tensorflow-addons
import tensorflow_addons as tfa

     |████████████████████████████████| 1.1 MB 5.4 MB/s 


In [ ]:
import cv2
import numpy as np
import json
import os
import random
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Dropout, GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.resnet import ResNet50
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import pandas as pd
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import seaborn as sns
import shutil
import keras
from PIL import Image
from matplotlib import image
from sklearn.cluster import AgglomerativeClustering
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.optimizers import Adam

In [ ]:
path = '/content/drive/MyDrive/CV_Images'

In [ ]:
def data_loader(path, data_name, img_size): # data_name = 'cropped_train', # img_size = 128 
  all_images = []
  # all_files = ['1040','1019', '1015', '1004', '1005','1039','1017','1021']
  all_files = ['1040','1019', '1015', '1004', '1005','1039','1017','1021','1001','1007','1018']
  for label_folder in all_files:
    inside_path = path+'/'+ label_folder +'/'+ data_name
    fold = os.listdir(inside_path)
    for img  in fold:
      ext = os.path.splitext(img)[-1]
      if ext in ['.jpg', '.JPG']: # only jpg files no json files
        try:
          img_arr = cv2.imread(os.path.join(inside_path, img))
          #print(img_arr.shape)
          new_arr = cv2.resize(img_arr, (img_size, img_size)) 
          all_images.append([new_arr, label_folder])  
        except:
          pass 
    print(label_folder,data_name,len(all_images))
    random.shuffle(all_images)
  return all_images


# rename cropped_train to folder name which you want to run
# img_train = data_loader(path, 'train', 224)
# img_test_cr = data_loader(path, 'cropped_test', 224)
# img_test_uncr = data_loader(path, 'uncropped_test', 224)

In [ ]:
# label_dict = {'1040':0, '1019':1, '1015':2, '1004':3, '1005':4, 
#               '1039':5, '1017':6, '1021':7}#, '1018':8}  #'1008':8, # add the label in the dictionary #'1004':0, '1005':1, 

label_dict = {'1040':0, '1019':1, '1015':2, '1004':3, '1005':4, '1039':5, '1017':6,'1021':7,'1001':8,'1007':9,'1018':10} 


classes  = 11 #7 #6 #5 #2 # 4
def preprocesing(data, num_class):
  X, y = [], []
  for features, label in data:
    X.append(features)                 
    y.append(label)  
  X = np.array(X).reshape(-1, 224, 224, 3)
  encoding =np.vectorize(label_dict.get)(y) # label encoding
  y = np.array(encoding)
  y = tf.keras.utils.to_categorical(y, num_classes=num_class) # one hot encoding
  print("The shape of the dataset:", X.shape)
  print("The shape of the label dataset:", y.shape)
  #X = X/255
  return X, y

# X_train, y_train = preprocesing(img_train,classes)
# X_test_cr, y_test_cr = preprocesing(img_test_cr,classes)
# X_test_uncr, y_test_uncr = preprocesing(img_test_uncr,classes)

In [ ]:
# np.save('/content/drive/MyDrive/CV_Images_npy_11/X_train', X_train)
# np.save('/content/drive/MyDrive/CV_Images_npy_11/y_train', y_train)
# np.save('/content/drive/MyDrive/CV_Images_npy_11/X_test_cr', X_test_cr)
# np.save('/content/drive/MyDrive/CV_Images_npy_11/y_test_cr', y_test_cr)
# np.save('/content/drive/MyDrive/CV_Images_npy_11/X_test_uncr', X_test_uncr)
# np.save('/content/drive/MyDrive/CV_Images_npy_11/y_test_uncr', y_test_uncr)

# Load data from npy files

X_train = np.load('/content/drive/MyDrive/CV_Images_npy_11/X_train.npy')
y_train = np.load('/content/drive/MyDrive/CV_Images_npy_11/y_train.npy')
X_test_cr = np.load('/content/drive/MyDrive/CV_Images_npy_11/X_test_cr.npy')
y_test_cr = np.load('/content/drive/MyDrive/CV_Images_npy_11/y_test_cr.npy')
X_test_uncr = np.load('/content/drive/MyDrive/CV_Images_npy_11/X_test_uncr.npy')
y_test_uncr = np.load('/content/drive/MyDrive/CV_Images_npy_11/y_test_uncr.npy')

In [ ]:
X_train = preprocess_input(X_train)
X_test_cr = preprocess_input(X_test_cr)
X_test_uncr = preprocess_input(X_test_uncr)

In [ ]:
He_initializer = tf.keras.initializers.HeNormal()
def getVGG19Model(lastFourTrainable=False):
  #vgg_model = VGG16(weights='imagenet', input_shape=X_train[0].shape, include_top=True)
  vgg_model = VGG19(weights='imagenet', input_shape=X_train[0].shape, include_top=True)

  # Make all layers untrainable
  for layer in vgg_model.layers[:]:
      layer.trainable = False

  # Add fully connected layer which have 1024 neuron to VGG-19 model
  output = vgg_model.get_layer('fc2').output
  output = Flatten(name='new_flatten')(output)
  output = Dense(units=1024, activation='relu', name='new_fc',kernel_initializer= He_initializer)(output)
  output = Dense(units=11, activation='softmax',kernel_initializer= He_initializer)(output)
  vgg_model = Model(vgg_model.input, output)

  # Make last 4 layers trainable if lastFourTrainable == True
  if lastFourTrainable == True:
    vgg_model.get_layer('block5_conv4').trainable = True
    vgg_model.get_layer('fc1').trainable = True
    vgg_model.get_layer('fc2').trainable = True
    vgg_model.get_layer('new_fc').trainable = True
  
  # optimizers = [
  #    tf.keras.optimizers.Adam(learning_rate=1e-6),
  #    tf.keras.optimizers.Adam(learning_rate=1e-3)]

  # optimizers_and_layers = [(optimizers[1], vgg_model.layers[-1]),(optimizers[1], vgg_model.layers[-2]),
  #                           (optimizers[0], vgg_model.layers[-4]), (optimizers[0], vgg_model.layers[-5]),
  #                           (optimizers[0], vgg_model.layers[-8])]
  # optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

  


  # Compile VGG-19 model
  vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  # vgg_model.summary()

  return vgg_model

# Without Augmentation

In [ ]:
vgg19_model = getVGG19Model()

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

vgg19_model.fit(X_train, y_train, batch_size=50, epochs=40, validation_data=(X_test_uncr, y_test_uncr), callbacks=[es])

Epoch 1/40
18/18 [==============================] - 21s 1s/step - loss: 7.6588 - accuracy: 0.5916 - val_loss: 3.2791 - val_accuracy: 0.5982
Epoch 2/40
18/18 [==============================] - 15s 838ms/step - loss: 0.6843 - accuracy: 0.8544 - val_loss: 1.2448 - val_accuracy: 0.7055
Epoch 3/40
18/18 [==============================] - 15s 838ms/step - loss: 0.1022 - accuracy: 0.9727 - val_loss: 0.9334 - val_accuracy: 0.7761
Epoch 4/40
18/18 [==============================] - 15s 834ms/step - loss: 0.0164 - accuracy: 0.9932 - val_loss: 0.7594 - val_accuracy: 0.8313
Epoch 5/40
18/18 [==============================] - 15s 820ms/step - loss: 0.0046 - accuracy: 0.9989 - val_loss: 0.9135 - val_accuracy: 0.7914
Epoch 6/40
18/18 [==============================] - 15s 822ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.8742 - val_accuracy: 0.8006
Epoch 7/40
18/18 [==============================] - 15s 820ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.9241 - val_accuracy: 0.7883
Ep

In [ ]:
y_pred_cr = vgg19_model.predict(X_test_cr)
y_pred_uncr = vgg19_model.predict(X_test_uncr)

_, acc = vgg19_model.evaluate(X_test_cr, y_test_cr, verbose=0)
print('> %.3f' % (acc * 100.0))

_, acc = vgg19_model.evaluate(X_test_uncr, y_test_uncr, verbose=0)
print('> %.3f' % (acc * 100.0))

> 90.491
> 83.129


# Default Augmentation

In [ ]:
X_train_dg = ImageDataGenerator()
X_train_dg.fit(X_train)

In [ ]:
vgg19_model = getVGG19Model()

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

vgg19_model.fit(X_train_dg.flow(X_train, y_train), batch_size=50, epochs=40, validation_data=(X_test_uncr, y_test_uncr), callbacks=[es])

Epoch 1/40
28/28 [==============================] - 16s 544ms/step - loss: 4.3412 - accuracy: 0.6633 - val_loss: 1.3884 - val_accuracy: 0.7055
Epoch 2/40
28/28 [==============================] - 14s 492ms/step - loss: 0.1170 - accuracy: 0.9636 - val_loss: 1.3424 - val_accuracy: 0.6963
Epoch 3/40
28/28 [==============================] - 14s 508ms/step - loss: 0.0419 - accuracy: 0.9841 - val_loss: 1.0076 - val_accuracy: 0.7147
Epoch 4/40
28/28 [==============================] - 14s 500ms/step - loss: 0.0085 - accuracy: 0.9989 - val_loss: 1.0190 - val_accuracy: 0.7209
Epoch 5/40
28/28 [==============================] - 14s 500ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 1.0071 - val_accuracy: 0.7362
Epoch 6/40
28/28 [==============================] - 14s 499ms/step - loss: 7.7611e-04 - accuracy: 1.0000 - val_loss: 1.0128 - val_accuracy: 0.7485
Epoch 7/40
28/28 [==============================] - 14s 491ms/step - loss: 5.1704e-04 - accuracy: 1.0000 - val_loss: 1.0504 - val_accuracy

In [ ]:
y_pred_cr = vgg19_model.predict(X_test_cr)
y_pred_uncr = vgg19_model.predict(X_test_uncr)

_, acc = vgg19_model.evaluate(X_test_cr, y_test_cr, verbose=0)
print('> %.3f' % (acc * 100.0))

_, acc = vgg19_model.evaluate(X_test_uncr, y_test_uncr, verbose=0)
print('> %.3f' % (acc * 100.0))

> 91.104
> 74.847


# Augmentation 2

In [ ]:
X_train_dg = ImageDataGenerator(horizontal_flip=True)
X_train_dg.fit(X_train)

In [ ]:
vgg19_model = getVGG19Model()

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

vgg19_model.fit(X_train_dg.flow(X_train, y_train), batch_size=50, epochs=40, validation_data=(X_test_uncr, y_test_uncr), callbacks=[es])

Epoch 1/40
28/28 [==============================] - 99s 555ms/step - loss: 4.4007 - accuracy: 0.5904 - val_loss: 2.0983 - val_accuracy: 0.6411
Epoch 2/40
28/28 [==============================] - 14s 502ms/step - loss: 0.1355 - accuracy: 0.9647 - val_loss: 1.3685 - val_accuracy: 0.7147
Epoch 3/40
28/28 [==============================] - 14s 502ms/step - loss: 0.0225 - accuracy: 0.9943 - val_loss: 1.0976 - val_accuracy: 0.7546
Epoch 4/40
28/28 [==============================] - 14s 501ms/step - loss: 0.0248 - accuracy: 0.9886 - val_loss: 0.9994 - val_accuracy: 0.7638
Epoch 5/40
28/28 [==============================] - 14s 491ms/step - loss: 0.0237 - accuracy: 0.9932 - val_loss: 1.4669 - val_accuracy: 0.7270
Epoch 6/40
28/28 [==============================] - 14s 492ms/step - loss: 0.0147 - accuracy: 0.9943 - val_loss: 1.1664 - val_accuracy: 0.7393
Epoch 7/40
28/28 [==============================] - 14s 491ms/step - loss: 0.0387 - accuracy: 0.9886 - val_loss: 1.0455 - val_accuracy: 0.7515

In [ ]:
y_pred_cr = vgg19_model.predict(X_test_cr)
y_pred_uncr = vgg19_model.predict(X_test_uncr)

_, acc = vgg19_model.evaluate(X_test_cr, y_test_cr, verbose=0)
print('> %.3f' % (acc * 100.0))

_, acc = vgg19_model.evaluate(X_test_uncr, y_test_uncr, verbose=0)
print('> %.3f' % (acc * 100.0))

> 90.184
> 76.380


# Augmentation 3

In [ ]:
X_train_dg = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
X_train_dg.fit(X_train)

In [ ]:
vgg19_model = getVGG19Model()

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

vgg19_model.fit(X_train_dg.flow(X_train, y_train), batch_size=50, epochs=40, validation_data=(X_test_uncr, y_test_uncr), callbacks=[es])

Epoch 1/40
28/28 [==============================] - 20s 667ms/step - loss: 4.4272 - accuracy: 0.1968 - val_loss: 7.5494 - val_accuracy: 0.1196
Epoch 2/40
28/28 [==============================] - 19s 674ms/step - loss: 1.3628 - accuracy: 0.5802 - val_loss: 8.3118 - val_accuracy: 0.2025
Epoch 3/40
28/28 [==============================] - 20s 714ms/step - loss: 0.8841 - accuracy: 0.7941 - val_loss: 7.7359 - val_accuracy: 0.2975
Epoch 4/40
28/28 [==============================] - 18s 640ms/step - loss: 0.7035 - accuracy: 0.8168 - val_loss: 8.7657 - val_accuracy: 0.3252
Epoch 5/40
28/28 [==============================] - 18s 637ms/step - loss: 0.5222 - accuracy: 0.8874 - val_loss: 9.2549 - val_accuracy: 0.3374
Epoch 6/40
28/28 [==============================] - 18s 648ms/step - loss: 0.4552 - accuracy: 0.8874 - val_loss: 9.1268 - val_accuracy: 0.3681
Epoch 7/40
28/28 [==============================] - 18s 671ms/step - loss: 0.3844 - accuracy: 0.9067 - val_loss: 9.3682 - val_accuracy: 0.3712

In [ ]:
y_pred_cr = vgg19_model.predict(X_test_cr)
y_pred_uncr = vgg19_model.predict(X_test_uncr)

_, acc = vgg19_model.evaluate(X_test_cr, y_test_cr, verbose=0)
print('> %.3f' % (acc * 100.0))

_, acc = vgg19_model.evaluate(X_test_uncr, y_test_uncr, verbose=0)
print('> %.3f' % (acc * 100.0))

> 40.491
> 41.104


# Augmentation 4

In [ ]:
X_train_dg = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
X_train_dg.fit(X_train)

In [ ]:
vgg19_model = getVGG19Model()

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

vgg19_model.fit(X_train_dg.flow(X_train, y_train), batch_size=50, epochs=40, validation_data=(X_test_uncr, y_test_uncr), callbacks=[es])

Epoch 1/40
28/28 [==============================] - 19s 645ms/step - loss: 3.6818 - accuracy: 0.6098 - val_loss: 2.6046 - val_accuracy: 0.5521
Epoch 2/40
28/28 [==============================] - 17s 614ms/step - loss: 0.2826 - accuracy: 0.9283 - val_loss: 1.2066 - val_accuracy: 0.7270
Epoch 3/40
28/28 [==============================] - 17s 607ms/step - loss: 0.1396 - accuracy: 0.9511 - val_loss: 2.1096 - val_accuracy: 0.6779
Epoch 4/40
28/28 [==============================] - 17s 610ms/step - loss: 0.1318 - accuracy: 0.9522 - val_loss: 1.6995 - val_accuracy: 0.7239
Epoch 5/40
28/28 [==============================] - 17s 622ms/step - loss: 0.0930 - accuracy: 0.9704 - val_loss: 1.2397 - val_accuracy: 0.7577
Epoch 6/40
28/28 [==============================] - 17s 618ms/step - loss: 0.0801 - accuracy: 0.9738 - val_loss: 1.2800 - val_accuracy: 0.7393
Epoch 7/40
28/28 [==============================] - 17s 617ms/step - loss: 0.0394 - accuracy: 0.9852 - val_loss: 1.3126 - val_accuracy: 0.7301

In [ ]:
y_pred_cr = vgg19_model.predict(X_test_cr)
y_pred_uncr = vgg19_model.predict(X_test_uncr)

_, acc = vgg19_model.evaluate(X_test_cr, y_test_cr, verbose=0)
print('> %.3f' % (acc * 100.0))

_, acc = vgg19_model.evaluate(X_test_uncr, y_test_uncr, verbose=0)
print('> %.3f' % (acc * 100.0))

> 90.491
> 75.767


# Augmentation 4 with discriminative learning

In [ ]:
He_initializer = tf.keras.initializers.HeNormal()
def getVGG19Model(lastFourTrainable=False):
  #vgg_model = VGG16(weights='imagenet', input_shape=X_train[0].shape, include_top=True)
  vgg_model = VGG19(weights='imagenet', input_shape=X_train[0].shape, include_top=True)

  # Make all layers untrainable
  for layer in vgg_model.layers[:]:
      layer.trainable = False

  # Add fully connected layer which have 1024 neuron to VGG-19 model
  output = vgg_model.get_layer('fc2').output
  output = Flatten(name='new_flatten')(output)
  output = Dense(units=1024, activation='relu', name='new_fc',kernel_initializer= He_initializer)(output)
  output = Dense(units=11, activation='softmax',kernel_initializer= He_initializer)(output)
  vgg_model = Model(vgg_model.input, output)

  # Make last 4 layers trainable if lastFourTrainable == True
  if lastFourTrainable == True:
    vgg_model.get_layer('block5_conv4').trainable = True
    vgg_model.get_layer('fc1').trainable = True
    vgg_model.get_layer('fc2').trainable = True
    vgg_model.get_layer('new_fc').trainable = True
  
  optimizers = [
     tf.keras.optimizers.Adam(learning_rate=1e-6),
     tf.keras.optimizers.Adam(learning_rate=1e-3)]

  optimizers_and_layers = [(optimizers[1], vgg_model.layers[-1]),(optimizers[1], vgg_model.layers[-2]),
                            (optimizers[0], vgg_model.layers[-4]), (optimizers[0], vgg_model.layers[-5]),
                            (optimizers[0], vgg_model.layers[-8])]
  optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

  


  # Compile VGG-19 model
  vgg_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  # vgg_model.summary()

  return vgg_model

In [ ]:
X_train_dg = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
X_train_dg.fit(X_train)

In [ ]:
vgg19_model = getVGG19Model(lastFourTrainable=True)

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10,  restore_best_weights=True)

vgg19_model.fit(X_train_dg.flow(X_train, y_train), batch_size=50, epochs=40, validation_data=(X_test_uncr, y_test_uncr), callbacks=[es])

Epoch 1/40
28/28 [==============================] - 21s 670ms/step - loss: 4.4334 - accuracy: 0.5563 - val_loss: 2.6427 - val_accuracy: 0.4509
Epoch 2/40
28/28 [==============================] - 18s 641ms/step - loss: 0.2322 - accuracy: 0.9249 - val_loss: 1.1094 - val_accuracy: 0.7147
Epoch 3/40
28/28 [==============================] - 18s 639ms/step - loss: 0.1103 - accuracy: 0.9625 - val_loss: 0.9921 - val_accuracy: 0.7301
Epoch 4/40
28/28 [==============================] - 18s 644ms/step - loss: 0.0419 - accuracy: 0.9909 - val_loss: 1.3828 - val_accuracy: 0.7301
Epoch 5/40
28/28 [==============================] - 18s 637ms/step - loss: 0.0345 - accuracy: 0.9909 - val_loss: 1.4801 - val_accuracy: 0.7209
Epoch 6/40
28/28 [==============================] - 18s 640ms/step - loss: 0.0436 - accuracy: 0.9875 - val_loss: 0.9017 - val_accuracy: 0.7607
Epoch 7/40
28/28 [==============================] - 18s 631ms/step - loss: 0.0243 - accuracy: 0.9920 - val_loss: 1.3756 - val_accuracy: 0.7331

In [ ]:
y_pred_cr = vgg19_model.predict(X_test_cr)
y_pred_uncr = vgg19_model.predict(X_test_uncr)

_, acc = vgg19_model.evaluate(X_test_cr, y_test_cr, verbose=0)
print('> %.3f' % (acc * 100.0))

_, acc = vgg19_model.evaluate(X_test_uncr, y_test_uncr, verbose=0)
print('> %.3f' % (acc * 100.0))

> 92.638
> 80.061
